In [8]:
# ---------------------------------------------------------------
# Nondirectional Brain–Heart correlation: EEG power–Heart Correlation and FDR Analysis
# ---------------------------------------------------------------
# 1. Loads EEG and ECG time series for children.
# 2. Groups subjects by age.
# 3. Computes EEG–heart Pearson correlations per emotion and age group.
# 4. Applies FDR correction to p-values.
# 5. Outputs correlation tables with r and corrected p-values.
# ---------------------------------------------------------------

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests

# ----------------------- Configuration -----------------------
child_info_path = "/all_children_info.csv"
ECG_TS_folder   = "/movie_ECG_TS/"
EEG_TS_folder   = "/EEG_pow_region/"

regions    = ['Frontal', 'Central', 'Posterior']
bands      = ['delta', 'theta', 'alpha']
emotions   = ['fear', 'happy']
heart_vars = ["HF", "LF", "IBI"]

# ----------------------- Age grouping function -----------------------
def assign_age_group(age_month):
    """Categorize children into age groups (in months)."""
    if 60 <= age_month <= 83:
        return "G1 (5-6y)"
    elif 84 <= age_month <= 107:
        return "G2 (7-8y)"
    elif 108 <= age_month <= 131:
        return "G3 (9-10y)"
    else:
        return np.nan

# ----------------------- Load subject metadata -----------------------
child_info = pd.read_csv(child_info_path)
child_info["Age_group"] = child_info["Age_month"].apply(assign_age_group)

# ----------------------- Compute correlations per age group -----------------------
results = []
for group in ["G1 (5-6y)", "G2 (7-8y)", "G3 (9-10y)"]:
    group_df = child_info[child_info["Age_group"] == group]

    for emo in emotions:
        eeg_group_data = {}  # key: (band, region)
        ecg_group_data = {}  # key: heart_var

        for _, row in group_df.iterrows():
            ID = row["ID"]
            age = row["Age_month"]

            # ----- Load ECG data -----
            ecg_file = os.path.join(ECG_TS_folder, f"{ID}_movie_ECG_TS.mat")
            ecg = loadmat(ecg_file)
            neutral_rr = np.mean(ecg["neutral_RR"])

            hf = np.log(ecg[f"{emo}_hf"].squeeze()[:176] + 1) - np.log(np.mean(ecg["neutral_hf"][:176]) + 1)
            lf = np.log(ecg[f"{emo}_lf"].squeeze()[:176] + 1) - np.log(np.mean(ecg["neutral_lf"][:176]) + 1)
            rr_data = ecg[f"{emo}_RR_res"][0, 0].squeeze()[:704]
            ibi_full = np.mean(rr_data.reshape(-1, 4), axis=1) / 1000 - neutral_rr

            for hv, data in zip(heart_vars, [hf, lf, ibi_full]):
                ecg_group_data.setdefault(hv, []).append(data)

            # ----- Load EEG data -----
            eeg_file = os.path.join(EEG_TS_folder, f"ID {ID} Age {age} EEG_pow.mat")
            mat = loadmat(eeg_file, struct_as_record=False, squeeze_me=True)
            EEG = mat["EEG_pow_region"]

            for band in bands:
                eeg_band_data = getattr(getattr(EEG, emo), band)
                eeg_neutral_data = getattr(getattr(EEG, "neutral"), band)

                for region in regions:
                    eeg_ts = np.log(getattr(eeg_band_data, region)[:176] + 1) - np.mean(
                        np.log(getattr(eeg_neutral_data, region)[:176] + 1)
                    )
                    eeg_group_data.setdefault((band, region), []).append(eeg_ts)

        # ----- Average across subjects -----
        ecg_avg = {hv: np.mean(np.vstack(trials), axis=0) for hv, trials in ecg_group_data.items()}
        eeg_avg = {key: np.mean(np.vstack(trials), axis=0) for key, trials in eeg_group_data.items()}

        # ----- Compute correlation and p-values -----
        corr_matrix = np.zeros((len(bands) * len(regions), len(heart_vars)))
        pval_matrix = np.ones_like(corr_matrix)

        for i, (band, region) in enumerate([(b, r) for b in bands for r in regions]):
            for j, hv in enumerate(heart_vars):
                r_val, p_val = pearsonr(eeg_avg[(band, region)], ecg_avg[hv])
                corr_matrix[i, j] = r_val
                pval_matrix[i, j] = p_val

        idx_names = [f"{b}_{r}" for b in bands for r in regions]
        corr_df  = pd.DataFrame(corr_matrix, index=idx_names, columns=heart_vars)
        pval_df  = pd.DataFrame(pval_matrix, index=idx_names, columns=heart_vars)

        results.append({
            "Age_group": group,
            "Emotion": emo,
            "Corr_df": corr_df,
            "Pval_df": pval_df,
        })

# ----------------------- FDR correction and table summary -----------------------
rows = []
for item in results:
    ageg = item["Age_group"]
    emo  = item["Emotion"]
    r_df = item["Corr_df"]
    p_df = item["Pval_df"]

    for eeg_var in r_df.index:
        for hv in r_df.columns:
            rows.append([ageg, emo, eeg_var, hv, r_df.loc[eeg_var, hv], p_df.loc[eeg_var, hv]])

df_all = pd.DataFrame(rows, columns=["Age_group", "Condition", "EEG", "Heart", "r", "p"])

# ----- Apply FDR correction -----
def fdr_group(g):
    pvals = g["p"].values
    if np.all(~np.isfinite(pvals)):
        g["p_FDR"] = np.nan
        return g
    _, p_fdr, _, _ = multipletests(pvals, method="fdr_bh")
    g["p_FDR"] = p_fdr
    return g

df_corr = df_all.groupby(["Age_group", "Condition"], group_keys=False).apply(fdr_group)

eeg_order = [f"{b}_{r}" for b in bands for r in regions]
df_corr["EEG"] = pd.Categorical(df_corr["EEG"], categories=eeg_order, ordered=True)
df_corr = df_corr.sort_values(["Age_group", "Condition", "EEG", "Heart"])
df_corr["r"] = df_corr["r"].round(3)
df_corr["p_FDR"] = df_corr["p_FDR"].round(3)